In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000
!unzip -q skin-cancer-mnist-ham10000.zip -d ham10000

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv("ham10000/hmnist_28_28_RGB.csv")

In [ ]:
for i in range(1,7):
  last_column = data.iloc[:, -1]
  valid_rows = data[(last_column == 0) | (last_column == i)]
  valid_rows.to_csv(f'{i}.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd

for i in range(1,7):
  fname=str(i)+'.csv'
  data=pd.read_csv(fname)
  print(data.shape)
  print(data[data.columns[-1]].value_counts())
  data=data.iloc[:,:]
  data=np.array(data)
  np.savetxt(fname,data,delimiter=',', fmt='%f')

In [ ]:
from scipy import stats

In [ ]:


import numpy as np

def sigtestfinal(j):
    fname=f"{j}.csv"
    df=np.genfromtxt(fname,delimiter=',')
    num_features = df.shape[1] - 1
    p_values = np.zeros(num_features)
    y=df[:, -1]
    in0=np.where(y==0)
    in1=np.where(y>0)
    for i in range(num_features):
        feature_values = df[:, i]
        _, p_values[i] = stats.ranksums(feature_values[in0], feature_values[in1])
    return p_values,df

In [ ]:
impf=[]
impf1=[]
for i in range(1,7):
    p,df=sigtestfinal(i)
    sigf=np.where(p<=0.05)
    impf.append(sigf[0])
    print(i,len(sigf[0]))
    data=np.concatenate((df[:,sigf[0]],df[:,-1].reshape(-1,1)), axis=1)
    fname=str(6+i)+'.csv'
    np.savetxt(fname,data, delimiter=',', fmt='%f')

In [ ]:
from sklearn.decomposition import PCA
for i in range(1,7):
    fname=str(i)+'.csv'
    df=np.genfromtxt(fname,delimiter=',')
    pca = PCA(n_components=0.99)
    pca.fit(df[:,0:-1])
    data=pca.transform(df[:,0:-1])
    data=np.concatenate((data,df[:,-1].reshape(-1,1)), axis=1)
    print(i,np.shape(data))
    fname=str(12+i)+'.csv'
    np.savetxt(fname,data, delimiter=',', fmt='%f')

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

In [ ]:
def fmodel(trdata):
  model=[]
  model.append(MultinomialNB())
  model.append(BernoulliNB())
  model.append(GaussianNB())
  model.append(DecisionTreeClassifier())
  model.append(LogisticRegression())
  model.append(KNeighborsClassifier())
  model.append(svm.SVC(kernel="linear",probability=True))
  model.append(svm.SVC(kernel="poly",probability=True))
  model.append(svm.SVC(kernel="rbf",probability=True))
  model.append(BaggingClassifier(MultinomialNB(),max_samples=0.5, max_features=0.5))
  model.append(BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5, max_features=0.5))
  model.append(BaggingClassifier(LogisticRegression(),max_samples=0.5, max_features=0.5))
  model.append(BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5))
  model.append(RandomForestClassifier(n_estimators=10))
  model.append(ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0))
  model.append(AdaBoostClassifier(n_estimators=10))
  model.append(GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,max_depth=1, random_state=0))
  model.append(MLPClassifier(solver='lbfgs', hidden_layer_sizes=(trdata.shape[1], 2), random_state=1,max_iter=1500))
  model.append(MLPClassifier(solver='sgd', hidden_layer_sizes=(trdata.shape[1], 2), random_state=1,max_iter=1500))
  model.append(MLPClassifier(solver='adam', hidden_layer_sizes=(trdata.shape[1], 2), random_state=1,max_iter=1500))
  return model


In [ ]:
from sklearn.preprocessing import MinMaxScaler
def normalizedata(X_train):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    return X_train

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(3)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import time
import os

def fmodel(x):
    model = []
    model.append(GaussianNB())
    model.append(LogisticRegression(max_iter=1000))
    model.append(KNeighborsClassifier())
    return model

def normalizedata(X):
    return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0) + 1e-8)



for i in range(1,19):
    start = time.time()
    print(f"\nProcessing Dataset {i}")
    fname=f"{i}.csv"
    data=np.genfromtxt(fname,delimiter=',')
    np.random.shuffle(data)
    predvalue=np.zeros((data.shape[0],21))
    predvalue1=np.zeros((data.shape[0],21))
    data[:,:-1]=normalizedata(data[:, :-1])
    for fold_idx, (train_index, test_index) in enumerate(kf.split(data)):
        trdata = data[train_index, :-1]
        tsdata = data[test_index, :-1]
        tract = data[train_index, -1]
        tsact = data[test_index, -1]

        if len(tract.shape) > 1:
            tract = tract.ravel()
        tract = tract.astype(np.float64)
        model=fmodel(trdata)
        for j in range(len(model)):
            mod = model[j]
            try:
                model_start = time.time()
                mod.fit(trdata, tract)
                predvalue[test_index, j] = mod.predict(tsdata)

                try:
                    predvalue1[test_index, j] = mod.predict_proba(tsdata)[:, 1]
                except:
                    pass

                print(f"✅ Finished model {j+1}/{len(model)} for dataset {i} on fold {fold_idx+1}")
                print(f"⏱ Model {j+1} took {time.time() - model_start:.2f} sec")
            except Exception as e:
                print(f"❌ Model {j+1} failed on dataset {i}: {e}")


        predvalue[test_index,20]=tsact
        predvalue1[test_index,20]=tsact
    """np.savetxt("pred.csv",predvalue, delimiter=',', fmt='%f')
    np.savetxt("predp.csv",predvalue1, delimiter=',', fmt='%f')"""
    np.savetxt(f"{i}pred.csv", predvalue, delimiter=',', fmt='%f')
    np.savetxt(f"{i}predp.csv", predvalue1, delimiter=',', fmt='%f')
    print(f"✅ Finished Dataset {i} in {time.time() - start:.2f} seconds")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import (
    f1_score, precision_score, recall_score,accuracy_score
    )

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
acv=np.zeros((72,20))
for i in range(0,18):
    fname= str(i+1)+"pred.csv"
    data1=np.genfromtxt(fname,delimiter=',')
    y1=data1[:, -1]
    unique_labels = np.unique(y1)
    if len(unique_labels) != 2:
        print(f"⚠️ Skipping dataset {i+1}: not binary labels: {unique_labels}")
        continue
    label_map = {unique_labels[0]: 0, unique_labels[1]: 1}
    y1 = np.vectorize(label_map.get)(y1)
    for j in range(0,20):
        pred_mapped = np.vectorize(label_map.get)(data1[:, j])
        acv[i, j] = accuracy_score(y1, pred_mapped)
        acv[18+i,j]=precision_score(y1, pred_mapped)
        acv[36+i,j]=recall_score(y1, pred_mapped)
        acv[54+i,j]=f1_score(y1, pred_mapped)
fname='acc.csv'
np.savetxt(fname,acv, delimiter=',', fmt='%f')

In [ ]:
from sklearn.metrics import roc_curve, auc
import numpy as np
rocv=np.zeros((18,20))
for i in range(0,18):
    fname= str(i+1)+"pred.csv"
    data=np.genfromtxt(fname,delimiter=',')
    y1=data[:,-1]
    unique_labels = np.unique(y1)

    if len(unique_labels) != 2:
        print(f"Skipping dataset {i+1}: not binary labels: {unique_labels}")
        continue


    label_map = {unique_labels[0]: 0, unique_labels[1]: 1}
    y1 = np.vectorize(label_map.get)(y1)
    for j in range(0,20):
        b = np.vectorize(label_map.get)(data[:, j])
        fpr, tpr, _ = roc_curve(y1, b)
        rocv[i,j]=auc(fpr, tpr)
fname='auc.csv'
np.savetxt(fname,rocv, delimiter=',', fmt='%f')

In [ ]:
!git clone https://github.com/raimarajan12/skin_cancer_ml.git
%cd  skin_cancer_ml
!mv /content/SKIN_cancer.ipynb /content/skin_cancer_ml/
from getpass import getpass
token = getpass("Enter your GitHub token: ")

!git config --global user.email "rajanraima12@email.com"
!git config --global user.name "raimarajan12"

!git add SKIN_cancer.ipynb
!git commit -m "Initial push of clean notebook"
push_url = f"https://{token}@github.com/raimarajan12/skin_cancer_ml.git"
!git push {push_url}


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'skin_cancer_ml': No such file or directory
[Errno 2] No such file or directory: 'skin_cancer_ml'
/content/skin_cancer_ml
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
mv: cannot stat '/content/SKIN_cancer.ipynb': No such file or directory
